In [1]:
# imports

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [1]:
import requests
import pandas as pd
import json

#using paris as an example 
lattitude = 48.8588548
longitude = 2.347035

def get_json_by_ll(lattitude,longitude):
    """
    this function takes two integers as input, lattitude adn longitude.
    it returns a json object to convert to dataframe
    """
    #construct the url witht he requests library functions
    url = "https://api.foursquare.com/v3/places/search"
    params = {
        "ll": str(lattitude)+","+str(longitude),
        "radius": 1000,
        "fields": "categories,rating,geocodes,categories,location,name,fsq_id,stats,price",
        "limit": 50
    }
    headers = {
        "accept": "application/json",
        "Authorization": "fsq3aB5c7PNhhqT23DewFyJ7Easfex4oG/ymaLVxSui+7VE="
    }
    
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    ratings = [place.get('rating', 0) for place in data.get('results', [])]
    return data


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [2]:
response = get_json_by_ll(lattitude,longitude)

def get_fs_data(jsonresponse):
    """
    this is a function that takes the json response and uses getter functions on the data structure 
    to return required details, it returns a list of details to be used in a dataframe
    """
    poi_details = [] # POI - point of interest 
    for poi in response.get('results', []):
        idd = poi.get('fsq_id', 0)
        name = poi.get('name', 'no data')
        rating = poi.get('rating', 0)
        location = poi.get('location', {}).get('formatted_address', 'no data')
        categories = poi.get('categories','no data')
        poi_info = {
            "id": idd,
            "name": name,
            "rating": rating,
            "location": location,
        }


        poi_details.append(poi_info)
    return poi_details
list_of_stuff = get_fs_data(response)

Put your parsed results into a DataFrame

In [3]:
# Create DataFrame
df1 = pd.DataFrame(list_of_stuff)

# Print the DataFrame
print(df1)

                          id  \
0   4b27effef964a5209a8c24e3   
1   4e0766702271dfa46baee474   
2   4d71280e1529a143391197a0   
3   4b8ebd9df964a5203c3433e3   
4   4adcda08f964a520ab3321e3   
5   4b5c7d1ff964a5205f3229e3   
6   540063e9498e81e53bf4f527   
7   4dbd336b6a23e294ba405cfa   
8   4b9bf8ebf964a520253c36e3   
9   54351b59498e123ee4afc7ff   
10  519a24a8498e55ea1824b30a   
11  4d79108e0ddba35dfac24a76   
12  57035c8b498ebcbc85d70c40   
13  4c862093e602b1f71304bd7a   
14  4bf41231e5eba59334341f90   
15  4b1fe208f964a520362a24e3   
16  4adcda09f964a520eb3321e3   
17  5113ebbce4b0d3c1022aef14   
18  58728968d4ab735c99ae0c5e   
19  52b5c225498eeda0297a855a   
20  4f979c8ae4b05465dae0714f   
21  4adcda11f964a520d33521e3   
22  4adcda05f964a5206c3221e3   
23  58e0f18f561ded7abcbbbd01   
24  51cfcf55498e81b280cea40e   
25  4b533b45f964a520509327e3   
26  4adcda0cf964a520c03421e3   
27  512534ece4b03a94f9a50313   
28  4b6bebdaf964a520a31c2ce3   
29  600af5ddd49d6e3b5b1abf05   
30  58d8

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [4]:
def yelp_json_by_name(name):
    API_KEY = "V6RfSQbnmYuJu5EBRgRikoCk-ScXUU38_M1rKj4CxeKGqmL0nGd1iLhkloqMVAsUK7uYrlpyE4OrIDRIrNO5lEPo60qcEEitGxX7tNBr8hRNu4RTHSKV_IztV2h7ZHYx"
    BASE_URL = "https://api.yelp.com/v3"
    term = "bike stations"
    location = name
    headers = {
        "Authorization": f"Bearer {API_KEY}"
    }
    params = {
        "term": term,
        "location": location,
        "radius": 1000
    }
    url = f"{BASE_URL}/businesses/search"
    response = requests.get(url, headers=headers, params=params)
    return response.json()



Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [5]:
data = yelp_json_by_name("Paris")

def get_yelp_data(json):   
    names = []
    ratings = []
    addresses = []
    phones = []
    
    for business in json['businesses']:
        names.append(business['name'])
        ratings.append(business.get('rating', 0))
        addresses.append(', '.join(business['location']['display_address']))
        phones.append(business.get('phone', 'no data'))
        
    data_dict = {
        'names': names,
        'ratings': ratings,
        'addresses': addresses,
        'phones': phones
    }
    
    return data_dict

result = get_yelp_data(data)
print(result)


{'names': ['Allo Vélo', 'Fat Tire Tours', 'Blue Fox Travel'], 'ratings': [4.0, 4.5, 5.0], 'addresses': ['2 rue Beauregard, 75002 Paris, France', '24 rue Edgar Faure, 75015 Paris, France', '75004 Paris, France'], 'phones': ['+33140353636', '+33182888096', '+33649323649']}


Put your parsed results into a DataFrame

In [6]:
dictionary_to_use = get_yelp_data(data)

df2 = pd.DataFrame({
    'name': dictionary_to_use['names'],
    'rating': dictionary_to_use['ratings'],
    'addresses': dictionary_to_use['addresses'],
    'phone': dictionary_to_use['phones']
})
print(df2)

              name  rating                                addresses  \
0        Allo Vélo     4.0    2 rue Beauregard, 75002 Paris, France   
1   Fat Tire Tours     4.5  24 rue Edgar Faure, 75015 Paris, France   
2  Blue Fox Travel     5.0                      75004 Paris, France   

          phone  
0  +33140353636  
1  +33182888096  
2  +33649323649  


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating

In [7]:
#When it comes to the questions i can clearly see that yelp has better data than foursquare for
#rating. The responses are detailed in their own ways plus achieving the data was not the same proces for either.
#I wouldnt pick one or the other im a programmer, if i need the informationt here is a way i can get it. I guess i prefer
#building params and ehaders from the requests library because its my most intuitive.

In [8]:

# Concatenate the dataframes
combined_df = pd.concat([df1, df2], ignore_index=True)
print(combined_df)


                          id  \
0   4b27effef964a5209a8c24e3   
1   4e0766702271dfa46baee474   
2   4d71280e1529a143391197a0   
3   4b8ebd9df964a5203c3433e3   
4   4adcda08f964a520ab3321e3   
5   4b5c7d1ff964a5205f3229e3   
6   540063e9498e81e53bf4f527   
7   4dbd336b6a23e294ba405cfa   
8   4b9bf8ebf964a520253c36e3   
9   54351b59498e123ee4afc7ff   
10  519a24a8498e55ea1824b30a   
11  4d79108e0ddba35dfac24a76   
12  57035c8b498ebcbc85d70c40   
13  4c862093e602b1f71304bd7a   
14  4bf41231e5eba59334341f90   
15  4b1fe208f964a520362a24e3   
16  4adcda09f964a520eb3321e3   
17  5113ebbce4b0d3c1022aef14   
18  58728968d4ab735c99ae0c5e   
19  52b5c225498eeda0297a855a   
20  4f979c8ae4b05465dae0714f   
21  4adcda11f964a520d33521e3   
22  4adcda05f964a5206c3221e3   
23  58e0f18f561ded7abcbbbd01   
24  51cfcf55498e81b280cea40e   
25  4b533b45f964a520509327e3   
26  4adcda0cf964a520c03421e3   
27  512534ece4b03a94f9a50313   
28  4b6bebdaf964a520a31c2ce3   
29  600af5ddd49d6e3b5b1abf05   
30  58d8